CCUBE MATCHING SYSTEM 1.0
* PETER BAZANOV
* PART1- SEMANTIC INTELLIGENCE BASED ON METRICS FOR MATCHING (FEATURE ENGINEERING)
  - VECTOR ENCODING
  - SIMILARITY DISTANCE & SEMANTIC
  - TOP N DECISION
* PART2 (IMROVEMENTS IN PROGRESS)- XGBOOST MACHINE LEARNING FOR DECISION INTELLIGENCE ON TARGET ID  (ERROR CORRECTION OF 1ST LOGIC)
- WORK ON RECOMMENDATIONS OF TARGET ID
- SEMI SUPERVISED MODEL
- WORK ON ERROR SUPERVISED USER CORRECTION
- WORK ON HiSTORY DB


In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
import matplotlib.pyplot as plt
from mahalanobis import Mahalanobis
%matplotlib inline
from fastnumbers import fast_real

Load the data and make sure that all of the empty strings have been filled.

In [2]:
import math
import pandas as pd
import scipy
import numpy as np

from category_encoders import *
from scipy import spatial
from sklearn.datasets import load_boston
from category_encoders import OneHotEncoder

DEB_FLAG=1

def geo_distance(origin, destination):

    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)

    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c
    return d


#weighted encoding based on sales of electric vehicle
gender_mapping = { "female" : 2,"company":5, "male" : 10}
lang_mapping = { "french":2, "english":5,"spanish" : 8,"portoguese" : 10 }
famstatus_mapping = { "single" : 1,"married" : 6}
profession_mapping = { "itprofessional" : 2,"taxidriver" : 20,"hotelmanager":6,"retailer":7,"biker" : 18,"bankmanager" : 13,"hotelmanager":17,"retailer":8,"architect":9,"propertymanager":10, "tourismmanager":11,"agriculturemanager":1,"none":-1    }
hobby_mapping = { "swimming" : 3,"cycling" : 10,"fitness":7,"cinema":1,"shopping" : 7,"diving" : 5,"meetupsocial":8,"investing":4,"carsocial":12,"garden":6, "none":-1 }
sport_mapping = { "Martial arts" : 7,"Watersports" : 2,"cycling":10,"dancing":5,"tennis" : 6,"motorsport" : 11,"Canoeing":9,"football":8,"basketball":4,"none":-1 }
travel_distance_mapping = { "low" : 100,"medium" :500,"high":1000}
age_group_mapping= { "young" : 20,"midage" :30,"senior":50}
lang_group_mapping= { "nonport" :0,"port_spain_group":1}
vehicle_life_group_mapping = { "Family" :3,"Mid":1,"VIP":10}
vehiclekind_mapping = { "BMWi3":1,"BMW330e":2,"Mercedes-BenzE300":3,"MitsubishiOutlander":4,"BMW530":5,"NISSAN LEAF":6,"JaguarPace":7,"Hyunday":8,"RenaultZOE":9,"TeslaModel3":10 }


def cmp_all( dataSetI, dataSetII):
    
    diflang=abs(dataSetI[0]-dataSetII[0])     
    difage=abs(dataSetI[1]-dataSetII[1])        
    difgender=abs(dataSetI[2]-dataSetII[2])     
    
    if dataSetI[0] ==dataSetII[0]:
        lang_decision=1
    else:
        lang_decision=0 if (dataSetI[0]<8 or dataSetII[0]<8 ) else 0.75
          
    age_decision = 0 if difage>=30 else (30-difage)/30        
         
    if difgender>=6:
        gender_decision=0.25
    else:
        gender_decision=(10-difgender)/10
                      
    
    res=lang_decision*age_decision*gender_decision
    
    
    return res



#Open Leads
bunch = pd.read_csv('input/leads_06.csv',dtype=str)
df = bunch     

df['gender']=df['gender'].map(gender_mapping)  
df['lang']=df['lang'].map(lang_mapping)  
df['famstatus']=df['famstatus'].map(famstatus_mapping)  
df['profession']=df['profession'].map(profession_mapping)  
df['hobby']=df['hobby'].map(hobby_mapping)  
df['sport']=df['sport'].map(sport_mapping)  
df['travel_distance']=df['travel_distance'].map(travel_distance_mapping)  
df['age_group']=df['age_group'].map(age_group_mapping)  
df['lang_group']=df['lang_group'].map(lang_group_mapping)  
df['vehicle_life_group']=df['vehicle_life_group'].map(vehicle_life_group_mapping)  
df['vehiclekind']=df['vehiclekind'].map(vehiclekind_mapping)  

#used Mapping or One Hot encoding
#enc = OrdinalEncoder(verbose=1, return_df=True, handle_unknown='return_nan')
#enc.fit(X)
#df = enc.transform(X)


# transform the dataset
numeric_dataset = df
numeric_dataset_file= pd.DataFrame(numeric_dataset)

numeric_dataset_file['TYPE']=bunch['TYPE']
numeric_dataset_file['userID']=bunch['userID']
numeric_dataset_file['UserLatitude']=bunch['UserLatitude']
numeric_dataset_file['UserLongitude']=bunch['UserLongitude']
numeric_dataset_file['age']=bunch['age']
numeric_dataset_file['target_id']=bunch['target_id']
numeric_dataset_file.to_csv('temp/leads_file_out_enc3.csv', header=True, index=False) 

#Open Agents
bunch= pd.read_csv('input/agents_06.csv',dtype=str)
#print ('****agents******')
#X=bunch[['gender','famstatus','lang','profession','hobby', 'sport','travel_distance', 'age_group','lang_group']]
#enc = OrdinalEncoder(verbose=1, return_df=True, handle_unknown='return_nan')
#enc.fit(X)
#df= enc.transform(X)

df=bunch

df['gender']=df['gender'].map(gender_mapping)  
df['lang']=df['lang'].map(lang_mapping)  
df['famstatus']=df['famstatus'].map(famstatus_mapping)  
df['profession']=df['profession'].map(profession_mapping)  
df['hobby']=df['hobby'].map(hobby_mapping)  
df['sport']=df['sport'].map(sport_mapping)  
df['travel_distance']=df['travel_distance'].map(travel_distance_mapping)  
df['age_group']=df['age_group'].map(age_group_mapping)  
df['lang_group']=df['lang_group'].map(lang_group_mapping)  


numeric_dataset = df

numeric_dataset_file= pd.DataFrame(numeric_dataset)
numeric_dataset_file['TYPE']=bunch['TYPE']
numeric_dataset_file['userID']=bunch['userID']
numeric_dataset_file['UserLatitude']=bunch['UserLatitude']
numeric_dataset_file['UserLongitude']=bunch['UserLongitude']
numeric_dataset_file['age']=bunch['age']
numeric_dataset_file['target_id']=bunch['target_id']
numeric_dataset_file['agent_experience']=bunch['agent_experience']
numeric_dataset_file['agent_location_group']=bunch['agent_location_group']
numeric_dataset_file['alpha']=bunch['alpha']
numeric_dataset_file['Exclusive']=bunch['Exclusive']

#One Hot encoding /Ordinal Save
numeric_dataset_file.to_csv('temp/ag_file_out_enc3.csv', header=True, index=False) 

df_leads = pd.read_csv('temp/leads_file_out_enc3.csv',dtype=str)
df_agents = pd.read_csv('temp/ag_file_out_enc3.csv',dtype=str)

df_leads.head()
df_agents.head()


if DEB_FLAG :
    #DISTANCE INTELLIGENCE
    np_df_leads=np.array(df_leads[['TYPE','userID']])
    Subdf_lead_user =df_leads[['userID']]
    Subdf_lead =df_leads[['lang','age','gender','famstatus','profession','hobby', 'sport', 'age_group', 'lang_group']]
    Subdf_lead_ag =df_agents[['userID']]   
    Subdf_ag =df_agents[['lang','age','gender','famstatus','profession','hobby', 'sport', 'age_group', 'lang_group']]
    
    outframe= pd.DataFrame(columns=[ 'LeadID', 'ContaMediaAccount', 'Distance', 'SemDistCosine', 'SemDistHamming','SemDistCorrel','DistVIPHamming','WeightSem','Final','Final2','LeadLong','LeadLat','AgentLong','AgentLat','StoryLead','StoryAgent','VIPLeadStory','VIPAgentStory' ])
    outframe_res=outframe
    
    dataSetI_distance=df_leads[['UserLatitude','UserLongitude']]
    dataSetI_VIPLEAD = df_leads[['vehicle_life_group','vehicle_life_group','vehicle_life_group','vehicle_life_group']]
    dataSetII_distance=df_agents[['UserLatitude','UserLongitude']]
    dataSetII_ALPHAAG = df_agents[['agent_experience', 'agent_location_group', 'alpha', 'Exclusive']]
    
    for i in range (0,len(Subdf_lead.iloc[:,0])):
        dataSetI = np.array((Subdf_lead.iloc[i,:]))
        dataSetI_d =np.array(dataSetI_distance.iloc[i,0:2])
        dataSetI_u = Subdf_lead_user.iloc[i,0:1]
        dataSetI_VIPLEAD_action=np.array(dataSetI_VIPLEAD.iloc[i,0:4])
        

        for j in range(0,len(Subdf_ag.iloc[:,0])):
            dataSetII = np.array((Subdf_ag.iloc[j,:]))
            dataSetII_d =np.array(dataSetI_distance.iloc[j,0:2])
            dataSetII_u = Subdf_lead_ag.iloc[j,0:1]
            dataSetII_ALPHAAG_action=np.array( dataSetII_ALPHAAG.iloc[j,0:4])           

            dataSetI = [float(i) for i in dataSetI]
            dataSetII = [float(i) for i in dataSetII]

            dataSetI_d = [float(i) for i in dataSetI_d]
            dataSetII_d = [float(i) for i in dataSetII_d]

            dataSetI_u = [int(i) for i in dataSetI_u]
            dataSetII_u = [int(i) for i in dataSetII_u]

            
            
                       
            #SEMANTIC COUSINE
            result = 1 - spatial.distance.cosine(dataSetI, dataSetII)               
            
            #SEMANTIC HAMMING
            result3= 0.1+1 - spatial.distance.hamming(dataSetI, dataSetII)    
            #GEO
            result2 = geo_distance(np.array(dataSetI_d), np.array(dataSetII_d))            
            
            #CANBERRA
            result5=  1/(spatial.distance.canberra(dataSetI, dataSetII)+0.001)    
            
            #WEIGHTED SEMANTIC
            
            result6=cmp_all(dataSetI, dataSetII)      
            if result6<0.5:
                result6=0  #dropout in case of dismatches gender, age, lang            
                
               
            #VIP HAMMING
            result4= 0.1+ 1 - spatial.distance.hamming(dataSetII_ALPHAAG_action, dataSetI_VIPLEAD_action)
            
            final=(500-result2)*result3*math.sqrt(result6)*(0.2* result+0.3*result3+0.1*result4+0.3*result5+result6*0.3)
            final2=result3*math.sqrt(result6)*(0.2* result+0.3*result3+0.1*result4+0.3*result5+result6*0.3)+ math.sqrt(500-result2)/1000
                                   
            #print('Lead id:',np.array(dataSetI_u),'ContaMediaAccount:',np.array(dataSetII_u),'Distance:', result2, 'SemDistCosine:',result, 'SemDistHamming:',result3,'SemDistCorrel:',result5,'DistVIPHamming:',result4,'WeightSem:',result6,'Final:',final, 'Final2:',final2,'LeadLong:', np.array(dataSetI_d),  'LeadLat:','AgentLong:', 'AgentLat:',np.array(dataSetII_d)  )
            if i % 10 ==1 and j % 20 ==1 :
                print('Lead id:',np.array(dataSetI_u),'ContaMediaAccount:',np.array(dataSetII_u),'Distance:', result2, 'Final:',final, 'Final2:',final2 )
            
                   
            outframe = outframe.append({'LeadID' :int(np.array(dataSetI_u)),'ContaMediaAccount': int(np.array(dataSetII_u)), 'Distance':result2, 'SemDistCosine':result, 'SemDistHamming':result3,'SemDistCorrel':result5, 'DistVIPHamming':result4,'WeightSem': result6,'Final':final,'Final2':final2,'LeadLat':np.array(dataSetI_d[0]),'LeadLong':np.array(dataSetI_d[1]),'AgentLat':np.array(dataSetII_d[0]),'AgentLong':np.array(dataSetII_d[1]) ,  'StoryLead':dataSetI,'StoryAgent':dataSetII, 'VIPLeadStory':dataSetI_VIPLEAD_action, 'VIPAgentStory':dataSetII_ALPHAAG_action  }, ignore_index=True)
            #outframe = outframe.append({'LeadID' :int(np.array(dataSetI_u)),'ContaMediaAccount': int(np.array(dataSetII_u)), 'Distance':result2, 'SemDistCosine':result, 'SemDistHamming':result3,'SemDistCorrel':result5, 'DistVIPHamming':result4,'Final':final,'Final2':final2,'LeadLat':np.array(dataSetI_d[0]),'LeadLong':np.array(dataSetI_d[1]),'AgentLat':np.array(dataSetII_d[0]),'AgentLong':np.array(dataSetII_d[1])  }, ignore_index=True)          

    
    outframe2=outframe
    resultfr=outframe.sort_values(by=['Final'],ascending=False).head(5*160)
    resultfr2=outframe2.sort_values(by=['Final2'],ascending=False).head(5*160)
    
    resultfr3=resultfr.sort_values(by=['LeadID','Final'],ascending=False)
    resultfr4=resultfr2.sort_values(by=['LeadID','Final2'],ascending=False)
  
        
 
    
    
    print('Saved')
    resultfr3.to_csv('out/file_out_075_dist_long.csv', header=True, index=True) 
    resultfr4.to_csv('out/file_out_075_sem_long.csv', header=True, index=True) 
    
    
 
    


Lead id: [10002] ContaMediaAccount: [102] Distance: 0.0 Final: 0.0 Final2: 0.022360679774997897
Lead id: [10002] ContaMediaAccount: [122] Distance: 103.44177429680256 Final: 0.0 Final2: 0.019913769751184668
Lead id: [10012] ContaMediaAccount: [102] Distance: 55.2318323740417 Final: 0.0 Final2: 0.021089527439607514
Lead id: [10012] ContaMediaAccount: [122] Distance: 63.937174170077725 Final: 210.0594677551169 Final2: 0.5026004737904812
Lead id: [10022] ContaMediaAccount: [102] Distance: 103.44177429680256 Final: 0.0 Final2: 0.019913769751184668
Lead id: [10022] ContaMediaAccount: [122] Distance: 0.0 Final: 0.0 Final2: 0.022360679774997897
Lead id: [10032] ContaMediaAccount: [102] Distance: 2.223898532891522 Final: 0.0 Final2: 0.022310896473855743
Lead id: [10032] ContaMediaAccount: [122] Distance: 105.26127269645991 Final: 0.0 Final2: 0.01986803279903524
Lead id: [10042] ContaMediaAccount: [102] Distance: 54.41266271933869 Final: 0.0 Final2: 0.02110893974790447
Lead id: [10042] ContaMed

Filter out any target with less than 5 entries

In [3]:
#MACHINE LEARNING LOGIC FOR SUPERVISED LEARNING ADJUSTMENT BASED ON TARGET COLUMNS AS FUNCTION FROM FEATURES, RULES
#APPLIED FOR LEADS


df=df_leads
df = df.fillna('None')
df = df.groupby('target_id').filter(lambda x: len(x) >= 5)

Build a function to get a dictionary mapper for the factors in the dataframe

In [4]:
def seqtodict(sequence, start=0):
    #transform a sequence to a set of pairs to make into a dictionary
    n = start
    for elem in sequence:
        yield elem,n
        n += 1    

def getItemDict(itemvalues):
    itemvalues = list(itemvalues.unique())
    itemdict = {}
    if 'None' not in itemvalues:
        itemvalues = ['None'] + itemvalues
    itemdict = dict( seqtodict(itemvalues))
    return itemdict

Create a dictionary for each one of the string inputs

In [5]:
itemdict = getItemDict(df['target_id'])

agecode=getItemDict(df['age'])
genderecode=getItemDict(df['gender'])
famstatuscode=getItemDict(df['famstatus'])
lang1code=getItemDict(df['lang'])

professioncode=getItemDict(df['profession'])
hobbycode=getItemDict(df['hobby'])

sportcode=getItemDict(df['sport'])
agegroupcode=getItemDict(df['age'])


f11code=getItemDict(df['UserLongitude'])
f12code=getItemDict(df['UserLatitude'])
f21code=getItemDict(df['userID'])

#decision assisting explainable
f22code=getItemDict(df['age_group'])
f23code=getItemDict(df['lang_group'])
f24code=getItemDict(df['vehicle_life_group'])

target_code=getItemDict(df['target_id'])

Map all of the factors to numerical values

In [6]:
def getCodeorNone(x,dictionary):
    output = 0
    try:
        output=dictionary[x]
    except:
        output=dictionary['None']
    return output

agecode=getItemDict(df['age'])

gendercode=getItemDict(df['gender'])

famstatuscode=getItemDict(df['famstatus'])
langcode=getItemDict(df['lang'])
sportcode=getItemDict(df['sport'])

df['age'] = df['age'].apply(lambda x: getCodeorNone(x,agecode))
df['gender'] = df['gender'].apply(lambda x: getCodeorNone(x,genderecode))
df['famstatus'] = df['famstatus'].apply(lambda x: getCodeorNone(x,famstatuscode))
df['lang'] = df['lang'].apply(lambda x: getCodeorNone(x,langcode))
df['sport'] = df['sport'].apply(lambda x: getCodeorNone(x,sportcode))
df['profession'] = df['profession'].apply(lambda x: getCodeorNone(x,professioncode))
df['hobby'] = df['hobby'].apply(lambda x: getCodeorNone(x,hobbycode))


df['feature11-code'] = df['UserLongitude'].apply(lambda x: getCodeorNone(x,f11code))
df['feature12-code'] = df['UserLatitude'].apply(lambda x: getCodeorNone(x,f12code))
df['feature21-code'] = df['userID'].apply(lambda x: getCodeorNone(x,f21code))
df['feature22-code'] = df['age_group'].apply(lambda x: getCodeorNone(x,f22code))
df['feature23-code'] = df['lang_group'].apply(lambda x: getCodeorNone(x,f23code))
df['feature24-code'] = df['vehicle_life_group'].apply(lambda x: getCodeorNone(x,f24code))

df['target_code'] = df['target_id'].apply(lambda x: getCodeorNone(x,itemdict))


Create test/train split

In [7]:
train, test = train_test_split(df, test_size=0.2, random_state=23,stratify=df['target_id'])

featurecolumns = ['age','gender','lang',
                      
                   'feature11-code',
                  'feature12-code',
                      
     'sport','famstatus',    
'feature22-code',
                  
                  'feature23-code',
                      'feature24-code'
                      
           
                  

            ]
'''


           
                    
                      
 '''                       
features_train = train[featurecolumns].values
labels_train = train['target_code'].values
features_test = test[featurecolumns].values
labels_test = test['target_code'].values

#print (features_train)
#print (features_test)

dtrain = xgb.DMatrix(features_train, label=labels_train)
dtest = xgb.DMatrix(features_test, label=labels_test)

In [8]:
# user defined evaluation function, return a pair metric_name, result
# This function takes the top 10 predictions and checks to see if the target label is in that set.
# The error is 1 - the fraction of rows where the label is in the top 10.
def evalerror(preds, dtrain,topNvalue=10):
    labels = dtrain.get_label()
    vals = np.argpartition(preds,-topNvalue)[:,-topNvalue:]
    error = 1 - float(vals.size - np.count_nonzero((vals.transpose() - labels).transpose()))/len(labels)
    # return a pair metric_name, result
    return 'error', error

In [9]:
## Train Model
num_round=300
param = {'max_depth': 5, 
    'eta': 0.2, 
    'silent': 0, 
    'gamma':2,
    'objective':'multi:softprob',
    'num_class':len(np.unique(labels_train))+1,
    'seed':32}
watchlist = [(dtest, 'eval'), (dtrain, 'train')]
bst = xgb.train(param, dtrain, num_round, watchlist, feval=evalerror, early_stopping_rounds=30)

[0]	eval-merror:0.842105	train-merror:0.708333	eval-error:0.210526	train-error:0.083333
Multiple eval metrics have been passed: 'train-error' will be used for early stopping.

Will train until train-error hasn't improved in 30 rounds.
[1]	eval-merror:0.947368	train-merror:0.708333	eval-error:0.210526	train-error:0.041667
[2]	eval-merror:0.894737	train-merror:0.694444	eval-error:0.315789	train-error:0.055556
[3]	eval-merror:0.894737	train-merror:0.708333	eval-error:0.263158	train-error:0.055556
[4]	eval-merror:0.894737	train-merror:0.694444	eval-error:0.263158	train-error:0.055556
[5]	eval-merror:0.894737	train-merror:0.708333	eval-error:0.263158	train-error:0.069444
[6]	eval-merror:0.894737	train-merror:0.708333	eval-error:0.210526	train-error:0.125
[7]	eval-merror:0.894737	train-merror:0.722222	eval-error:0.210526	train-error:0.125
[8]	eval-merror:0.894737	train-merror:0.736111	eval-error:0.210526	train-error:0.138889
[9]	eval-merror:0.894737	train-merror:0.736111	eval-error:0.105263	

Save the model.

In [10]:
pickle.dump(bst, open("xgb-python.model", "wb"))

Get predictions on the test set and get the "Traditional" accuracy - how likely was the top prediction to match the label.

In [11]:
preds = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)
test_labels = dtest.get_label()
test_labels[:10]
toppreds = np.argmax(preds,axis=1)
print("Accuracy: {}".format(accuracy_score(test_labels,toppreds)))

Accuracy: 0.05263157894736842


Check a random sample to double-check the top N prediction error.

In [12]:
print("Testing random entry:")
topNvalue =7

n = np.random.choice(len(test_labels))
print('n')
print(n)
a = preds[n,:]
print('a')
print(a)
topN=np.argpartition(preds,-topNvalue)[:,-topNvalue:]
print(topN)
print("Actual: {}".format(test_labels[n]))
print("Top: {}".format(toppreds[n]))
print("Top 7 Predictions: {}".format(topN[n,]))

# Get the accuracy in the top N results
acc=float(topN.size - np.count_nonzero((topN.transpose() - test_labels).transpose()))/len(test_labels)
print("In-sample accuracy: {}".format(acc))   

Testing random entry:
n
14
a
[0.05792449 0.06198056 0.06331748 0.06459703 0.06460196 0.06800327
 0.07338604 0.07066616 0.07338604 0.05973689 0.06800327 0.06800327
 0.07066616 0.06772409 0.06800327]
[[14  8  6  5 12  3  7]
 [ 5 12  7  9  6 13  8]
 [ 5 12  7  8  6 13  2]
 [ 5 12  7  8  6  2  3]
 [14  8  1  6 12  5  7]
 [10 14  8  6 12  5  7]
 [ 5 12  7  9  6  4  8]
 [ 5 12  7  1  6  3  8]
 [12  6  9  2  8 13  3]
 [ 5 12  7  8  6 13  3]
 [14  8  6  5 12  3  7]
 [14  8  6  5 12  3  7]
 [12  7  6  2  8 13  3]
 [ 5  1 12  7  6 13  8]
 [10 14  8  6 12  5  7]
 [ 5 12  7  9  6  4  8]
 [12  7  6  2  8 13  3]
 [14  9  8  6 12  5  7]
 [ 5 12  7  9  6  2  8]]
Actual: 4.0
Top: 6
Top 7 Predictions: [10 14  8  6 12  5  7]
In-sample accuracy: 0.5789473684210527


##### score_eval = pd.DataFrame({'topN':range(1,20)})
score_eval['error'] = score_eval['topN'].apply(lambda x: evalerror(preds,dtest,x)[1])
score_eval.plot(x='topN',y='error',kind='scatter')
plt.ylim(0,1)

Verify that the reload and scoring functions work.

In [13]:
bst2 = pickle.load(open("xgb-python.model", "rb"))
preds2 = bst2.predict(dtest, ntree_limit=bst2.best_ntree_limit)
1-evalerror(preds2,dtest)[1]

0.7894736842105263